# Preprocess

In [3]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


# Splitting into Train and Val

In [6]:
import splitfolders

# Path to the dataset directory
input_folder = 'kvasir-dataset'

# Output directories for train and test splits
output_folder = 'data'

# Split dataset into training (80%) and testing (20%) sets
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .2))


Copying files: 4000 files [00:53, 75.29 files/s] 


# Load

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = 'data/train'
test_dir = 'data/val'

# Image Data Generator with augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if you have more than 2 classes
)

# Test data generator
val_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if you have more than 2 classes
)


Found 3200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


# CNN Featire extraction

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the input shape
input_shape = (256, 256, 3)

# Building the CNN model using Input layer
cnn_model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # or 'softmax' for multiclass
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
cnn_model.fit(train_generator, validation_data=val_generator, epochs=10)


# Vision Transformer for Feature Extraction

In [ ]:
from transformers import ViTFeatureExtractor, TFViTModel
from PIL import Image
import numpy as np

# Load pre-trained ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
vit_model = TFViTModel.from_pretrained('google/vit-base-patch16-224')

def extract_vit_features(image_paths):
    features = []
    for image_path in image_paths:
        image = Image.open(image_path).resize((224, 224))
        inputs = feature_extractor(images=image, return_tensors="tf")
        outputs = vit_model(**inputs)
        features.append(outputs.last_hidden_state.numpy().flatten())
    return np.array(features)

# Example: Extract features from a list of image paths
image_paths = ['path/to/image1.jpg', 'path/to/image2.jpg', ...]
vit_features = extract_vit_features(image_paths)


# BiLSTM on Extracted Features

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional, Input
from tensorflow.keras.models import Model

inputs = Input(shape=(vit_features.shape[1],))
x = tf.expand_dims(inputs, axis=1)  # Reshape to (batch_size, timesteps, features)
x = Bidirectional(LSTM(64))(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

bilstm_model = Model(inputs, outputs)
bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train BiLSTM model
bilstm_model.fit(vit_features, labels, epochs=10, validation_split=0.2)


# SVM for Final Classification

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Extract features using the BiLSTM model
bilstm_features = bilstm_model.predict(vit_features)

# Train SVM
svm = SVC(kernel='linear')
svm.fit(bilstm_features, labels)

# Predict and evaluate
svm_predictions = svm.predict(bilstm_features)
print("SVM Accuracy:", accuracy_score(labels, svm_predictions))
